In [1]:
import base

import pandas as pd
import json
from collections import namedtuple
import re
import ngender
import time


In [2]:
# my private modules
import fund_scanner.database
import fund_scanner.readurl

In [16]:
#page=page_index,items_in_one_page

url = 'http://fund.eastmoney.com/Data/FundDataPortfolio_Interface.aspx?dt=14&mc=returnjson&ft=all&pn=1000000&pi=1&sc=abbname&st=asc'

body = fund_scanner.readurl.read_from_url(url)


In [17]:
insider = re.search('\[\[(.*?)\]\]', body).group(1)
insider = insider.replace('\'', '"')
insider = '{"main":[['+insider+']]}'
obj = json.loads(insider, object_hook=lambda d: namedtuple('X', d.keys())(*d.values()))
df =pd.DataFrame(obj.main)
df.sort_values(by=2)
#df = df.loc[:,[0,1,2,7,9]]
df.columns = [
    'managers_code', 'managers_name', 'manager_2_company', 'company_name',
    'manager_2_funds', 'funds_names', 'working_days', 
    '现任基金最佳回报', '现任基金代码', '现任基金名称', '现任基金资产总规模', '任职期间最佳回报'
]

In [19]:
df.head(1)

,managers_code,managers_name,manager_2_company,company_name,manager_2_funds,funds_names,working_days,现任基金最佳回报,现任基金代码,现任基金名称,现任基金资产总规模,任职期间最佳回报
0,30277862,艾小军,80000224,国泰基金,"000199,000218,001789,004253,020011,020021,0200...","国泰策略收益灵活配置混合,国泰黄金ETF联接A,国泰量化收益灵活配置混合,国泰黄金ETF联接...",1292,100.76%,510230,国泰上证180金融ETF,82.30亿元,100.76%


In [34]:
with fund_scanner.database.get_connection() as cursor:

    for index, row in df.iterrows():
        #print(row['净值日期'])
        sql = 'INSERT INTO managers '+\
        '(managers_name, managers_code, working_since_date) '+\
        'VALUES (%s,%s,DATE_SUB(Now(), interval %s day)) ON DUPLICATE KEY UPDATE '+\
        'managers_name=%s, working_since_date=DATE_SUB(Now(), interval %s day);'
        cursor.execute(
            sql, (
                row['managers_name'],
                row['managers_code'], 
                base.extract_float(row, 'working_days'),
                row['managers_name'], 
                base.extract_float(row, 'working_days')
            )
        )
        
        sql = 'select managers_id from managers where managers_code=%s limit 0,1;'
        cursor.execute(sql, (row['managers_code']))
        rs = cursor.fetchone()
        managers_id = rs['managers_id']

        sql = 'select companies_id from companies where companies_code=%s limit 0,1;'
        cursor.execute(sql, (row['manager_2_company']))
        rs = cursor.fetchone()
        companies_id = rs['companies_id']

        sql = 'Insert into companies_2_managers '+\
        '(managers_id, companies_id) '+\
        'values (%s, %s) ON DUPLICATE KEY UPDATE '+\
        'companies_id=%s;'
        cursor.execute(sql, (managers_id, companies_id, companies_id))

        
        sql = 'delete from managers_2_funds where managers_id=%s'
        cursor.execute(sql, (managers_id))
        in_p = ', '.join(list(map(lambda x: '%s', row['manager_2_funds'].split(','))))
        sql = 'select funds_id from funds where funds_code in (%s);'%in_p
        args = row['manager_2_funds'].split(',')
        cursor.execute(sql, args)
        rs = cursor.fetchall()
        for e in rs:
            sql = 'Insert into managers_2_funds '+\
            '(managers_id, funds_id) '+\
            'values (%s, %s);'
            cursor.execute(sql, (managers_id, e['funds_id']))







/home/liusida/anaconda3/lib/python3.5/site-packages/pymysql/cursors.py:323: Warning: (1265, "Data truncated for column 'working_since_date' at row 1")
  self._do_get_result()


In [26]:
row['manager_2_funds'].split(',')

['000199',
 '000218',
 '001789',
 '004253',
 '020011',
 '020021',
 '020022',
 '150215',
 '150216',
 '160224',
 '501016',
 '501019',
 '510230',
 '512660',
 '512880',
 '518800']